In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train= pd.read_csv('/content/drive/MyDrive/bangla-nlp/train.csv')
test=pd.read_csv('/content/drive/MyDrive/bangla-nlp/test.csv')

In [4]:
train.shape


(1356300, 2)

In [5]:
train=train.sample(frac=0.04, random_state=42)

In [6]:
train.shape

(54252, 2)

In [7]:
test.shape

(900, 2)

In [8]:
def drop_null_duplicate(df):
  df=df.drop_duplicates()
  df=df.dropna()
  return df

In [9]:
train = drop_null_duplicate(train)

In [10]:
test = drop_null_duplicate(test)

In [11]:
def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations

                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)

    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰⚽️✌�￰৷￰'''

    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out

In [12]:
train.head()

,Target,Input
95146,তিনি বলেন আমরা 1111111111 ভাগ অপহরণের ঘটনার রহ...,তিনি বলেন আনরা 1111111111 ভাগ অপহরণের ঘটনার রয...
1175169,সম্প্রতি বিশৃঙ্খলা অদক্ষতা আর দুর্নীতির অভিযোগ...,সপ্ড়তি বিশরিংকলা অদক্কতা সর দুরণতির অভিযোগে লা...
1166255,তৃতীয় পুরস্কার হেলথ মাস্টার নির্মাতা রাইয়াদ রা...,তৃতীয় পুরসকএ হেলত মাস্টার নিড়তা রাইয়াদ রাদ মেহ...
412205,এরপর তাঁকে আর কখনো দেখা যায়নি ।,এরপর রাঁকে আর কখনো দেখা যায়নি ।
1102814,এটি পরিবেশবান্ধব এ জন্যই যে এখানে ব্যবহার করা ...,এতি পরিবেশবণধব এ জনাই জে এখাণ েববহার করা বোতল ...


In [13]:
train['Target'] = train.Target.apply(lambda x: preprocessing(str(x)))
train['Input'] = train.Input.apply(lambda x: preprocessing(str(x)))

In [14]:
train.head()

,Target,Input
95146,তিনি বলেন আমরা ভাগ অপহরণের ঘটনার রহস্য উদ্ঘাট...,তিনি বলেন আনরা ভাগ অপহরণের ঘটনার রযসা উদ্ঘাটন...
1175169,সম্প্রতি বিশৃঙ্খলা অদক্ষতা আর দুর্নীতির অভিযোগ...,সপ্ড়তি বিশরিংকলা অদক্কতা সর দুরণতির অভিযোগে লা...
1166255,তৃতীয় পুরস্কার হেলথ মাস্টার নির্মাতা রাইয়াদ রা...,তৃতীয় পুরসকএ হেলত মাস্টার নিড়তা রাইয়াদ রাদ মেহ...
412205,এরপর তাঁকে আর কখনো দেখা যায়নি,এরপর রাঁকে আর কখনো দেখা যায়নি
1102814,এটি পরিবেশবান্ধব এ জন্যই যে এখানে ব্যবহার করা ...,এতি পরিবেশবণধব এ জনাই জে এখাণ েববহার করা বোতল ...


In [15]:
test.head()

,Target,Input
0,"পৃথিবীতে প্রতিটি মানুষই সুখী হতে চায়, কিন্তু স...","পৃথিবীতে প্রতিটি মানুষই সুখি হতে চায়, কিন্তু স..."
1,তোমার মনে সুখ আছে তাই তুমি এখন সুখী।,তোমার মনে সুত আছে তাই তুমি এখন সুখী।
2,আমাদের জীবনকে সুন্দর করতে প্রকৃতির রয়েছে অসীম ...,আমাদের জীবনের সুন্দর করতে প্রকৃতির রয়েছে অসীম ...
3,পৃথিবীর সবচেয়ে বড় গ্রন্থাগারটি হল কংগ্রেস গ্রন...,পৃথিবীর সবজেয়ে বড় গ্রন্থাগারটি হল কংগ্রেস গ্রন...
4,এই রকম গরমে ঠাণ্ডা পানি পান করার মজাই আলাদা।,এই কমর রগমে ঠাণ্ডা পানি পান করার মজাই আলাদা।


In [16]:
test['Target'] = test.Target.apply(lambda x: preprocessing(str(x)))
test['Input'] = test.Input.apply(lambda x: preprocessing(str(x)))

In [17]:
test.head()

,Target,Input
0,পৃথিবীতে প্রতিটি মানুষই সুখী হতে চায় কিন্তু সু...,পৃথিবীতে প্রতিটি মানুষই সুখি হতে চায় কিন্তু সু...
1,তোমার মনে সুখ আছে তাই তুমি এখন সুখী,তোমার মনে সুত আছে তাই তুমি এখন সুখী
2,আমাদের জীবনকে সুন্দর করতে প্রকৃতির রয়েছে অসীম ...,আমাদের জীবনের সুন্দর করতে প্রকৃতির রয়েছে অসীম ...
3,পৃথিবীর সবচেয়ে বড় গ্রন্থাগারটি হল কংগ্রেস গ্রন...,পৃথিবীর সবজেয়ে বড় গ্রন্থাগারটি হল কংগ্রেস গ্রন...
4,এই রকম গরমে ঠাণ্ডা পানি পান করার মজাই আলাদা,এই কমর রগমে ঠাণ্ডা পানি পান করার মজাই আলাদা


In [18]:
target_text_train = train['Target'].tolist()
input_text_train = train['Input'].tolist()


In [19]:
input_text_train[0:5]

['তিনি বলেন আনরা  ভাগ অপহরণের ঘটনার রযসা উদ্ঘাটন ও বিকতিমকে উদ্ধার করতে পেরেছি ',
 'সপ্ড়তি বিশরিংকলা অদক্কতা সর দুরণতির অভিযোগে লাবাদির নেতৃত বদলে নতুন অ্যাডহক কমিটি কড়া হয়ছ ',
 'তৃতীয় পুরসকএ হেলত মাস্টার নিড়তা রাইয়াদ রাদ মেহরব জামান চৌধুরী ও টানহম ঈশকাম ',
 'এরপর রাঁকে আর কখনো দেখা যায়নি ',
 'এতি পরিবেশবণধব এ জনাই জে এখাণ েববহার করা বোতল পরকরিতিরে মিশে যেত ']

In [20]:
target_text_test = test['Target'].tolist()
input_text_test = test['Input'].tolist()

In [21]:

input_text_test[0:5]

['পৃথিবীতে প্রতিটি মানুষই সুখি হতে চায় কিন্তু সুখি কী এবং তা কীভাবে পাওয়া যায় তা অনেকেই জানে না',
 'তোমার মনে সুত আছে তাই তুমি এখন সুখী',
 'আমাদের জীবনের সুন্দর করতে প্রকৃতির রয়েছে অসীম ভূমিকা',
 'পৃথিবীর সবজেয়ে বড় গ্রন্থাগারটি হল কংগ্রেস গ্রন্থাগার যা যুক্তরাষ্ট্রে রাজধানী ওয়াশিংটন ডিসিতে অবস্থিত',
 'এই কমর রগমে ঠাণ্ডা পানি পান করার মজাই আলাদা']

In [22]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(input_text_train + input_text_test)
tokenizer.fit_on_texts(target_text_train + target_text_test)



In [23]:
input_sequences_train = tokenizer.texts_to_sequences(input_text_train)
target_sequences_train = tokenizer.texts_to_sequences(target_text_train)
input_sequences_test = tokenizer.texts_to_sequences(input_text_test)
target_sequences_test = tokenizer.texts_to_sequences(target_text_test)

In [24]:
max_input_sequence_length = max([len(seq) for seq in input_sequences_train + input_sequences_test])
max_target_sequence_length = max([len(seq) for seq in target_sequences_train + target_sequences_test])
input_sequences_train_padded = pad_sequences(input_sequences_train, maxlen=max_input_sequence_length, padding='post')
target_sequences_train_padded = pad_sequences(target_sequences_train, maxlen=max_target_sequence_length, padding='post')
input_sequences_test_padded = pad_sequences(input_sequences_test, maxlen=max_input_sequence_length, padding='post')
target_sequences_test_padded = pad_sequences(target_sequences_test, maxlen=max_target_sequence_length, padding='post')

In [25]:
np.save('input_sequences_train.npy', input_sequences_train_padded)
np.save('target_sequences_train.npy', target_sequences_train_padded)
np.save('input_sequences_test.npy', input_sequences_test_padded)
np.save('target_sequences_test.npy', target_sequences_test_padded)
np.save('tokenizer.npy', tokenizer)

In [26]:
input_sequences_train_padded = np.load('input_sequences_train.npy')
target_sequences_train_padded = np.load('target_sequences_train.npy')

In [27]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
hidden_units = 128
latent_dim = 128

In [37]:
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(hidden_units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

In [38]:
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
attention = Attention()([decoder_outputs, encoder_outputs])
decoder_concat_input = tf.keras.layers.Concatenate(axis=-1)([decoder_outputs, attention])
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

In [39]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [40]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [41]:
num_epochs = 10
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
model.fit([input_sequences_train_padded, target_sequences_train_padded], target_sequences_train_padded,batch_size=16,callbacks=[early_stopping], epochs=num_epochs, verbose=1)

Epoch 1/10
3391/3391 [==============================] - 574s 168ms/step - loss: 1.4898 - accuracy: 0.8358
Epoch 2/10
3391/3391 [==============================] - 561s 166ms/step - loss: 0.5528 - accuracy: 0.9415
Epoch 3/10
3391/3391 [==============================] - 560s 165ms/step - loss: 0.3099 - accuracy: 0.9669
Epoch 4/10
3391/3391 [==============================] - 560s 165ms/step - loss: 0.1871 - accuracy: 0.9781
Epoch 5/10
3391/3391 [==============================] - 560s 165ms/step - loss: 0.1006 - accuracy: 0.9867
Epoch 6/10
3391/3391 [==============================] - 561s 165ms/step - loss: 0.0403 - accuracy: 0.9973
Epoch 7/10
3391/3391 [==============================] - 559s 165ms/step - loss: 0.0105 - accuracy: 0.9997
Epoch 8/10
3391/3391 [==============================] - 558s 164ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 9/10
3391/3391 [==============================] - 559s 165ms/step - loss: 8.9374e-04 - accuracy: 1.0000
Epoch 10/10
3391/3391 [===================

In [42]:
model.save('seq2seq_model_v2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
!pip install python-Levenshtein


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 23.8 MB/s eta 0:00:00


In [44]:
import Levenshtein
from nltk.translate.bleu_score import sentence_bleu
from tensorflow.keras.models import load_model
from nltk.translate.bleu_score import sentence_bleu

In [47]:
model = load_model('/content/seq2seq_model_v2.h5')

def calculate_wer(reference, hypothesis):
    if len(reference.split()) == 0:
        return 0 if len(hypothesis.split()) == 0 else 1
    return Levenshtein.distance(reference, hypothesis) / len(reference.split())

def calculate_cer(reference, hypothesis):
    if len(reference) == 0:
        return 0 if len(hypothesis) == 0 else 1
    return Levenshtein.distance(reference, hypothesis) / len(reference)

def calculate_bleu(reference, hypothesis):
    reference = [reference.split()]
    hypothesis = hypothesis.split()
    return sentence_bleu(reference, hypothesis)

def evaluate_model(model, tokenizer, input_sequences_test_padded, target_sequences_test_padded):
    references = []
    hypotheses = []
    for i in range(len(input_sequences_test_padded)):
        input_seq = input_sequences_test_padded[i:i+1]
        target_seq = target_sequences_test_padded[i:i+1]

        # Predict sequence
        predicted_seq = model.predict([input_seq, target_seq])[0]

        # Convert indices to words
        predicted_sentence = ' '.join([tokenizer.index_word[idx] for idx in np.argmax(predicted_seq, axis=1) if idx != 0])
        reference_sentence = ' '.join([tokenizer.index_word[idx] for idx in target_seq[0] if idx != 0])

        references.append(reference_sentence)
        hypotheses.append(predicted_sentence)

    return references, hypotheses

references, hypotheses = evaluate_model(model, tokenizer, input_sequences_test_padded, target_sequences_test_padded)

wer_scores = [calculate_wer(ref, hyp) for ref, hyp in zip(references, hypotheses)]
cer_scores = [calculate_cer(ref, hyp) for ref, hyp in zip(references, hypotheses)]
bleu_scores = [calculate_bleu(ref, hyp) for ref, hyp in zip(references, hypotheses)]

average_wer = sum(wer_scores) / len(wer_scores)
average_cer = sum(cer_scores) / len(cer_scores)
average_bleu = sum(bleu_scores) / len(bleu_scores)

print("Average WER:", average_wer)
print("Average CER:", average_cer)
print("Average BLEU Score:", average_bleu)

1/1 [==============================] - 0s 18ms/step
Average WER: 0.46909027191250546
Average CER: 0.07864318477783686
Average BLEU Score: 0.8366168205319402


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [48]:
print("Average WER:", average_wer)
print("Average CER:", average_cer)
print("Average BLEU Score:", average_bleu)

Average WER: 0.46909027191250546
Average CER: 0.07864318477783686
Average BLEU Score: 0.8366168205319402
